In [2]:
from __future__ import division
import numpy as np
from scipy.stats import norm

In [3]:
from datetime import datetime, timezone, timedelta
import os
import pandas as pd # data processing
import bitmex
import math
from tqdm.notebook import trange, tqdm # progress bar
import time
import numpy as np
# import warnings  
# warnings.filterwarnings('ignore')


test_mode = False
key = '1rw8DDZoBd3xGXOmo48lFfQv' if test_mode else '73rdj2LQf63ANuFczBOlINsS'
secret = 'ODoX-aeLDeLSPWidAtqNpnN70eWDKLgfdpdIUL6_fifPZhEu' if test_mode else 'TmIdFVEN-Muni_TAKpA0zghts7EHv5ytq5vZfrPQXRxM01x7'
client = bitmex.bitmex(test=test_mode, api_key=key, api_secret=secret)
binsizes = {"1m": 1, "5m": 5, "1h": 60, "4h": 240, "1d": 1440}
batch_size = 750

def get_all_bitmex(symbol, kline_size, save = False, start_time = None):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    oldest_point = start_time if start_time is not None else oldest_point
    print(newest_point, oldest_point)
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
#     data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

def minutes_of_new_data(symbol, kline_size, data, source):
    print(kline_size)
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

/Users/gabeheim/Documents/venvs/algo/lib/python3.6/site-packages/swagger_spec_validator/validator20.py:53: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  ref_dict['$ref'], '/'.join(path),


In [4]:
def mk_test(x, alpha=0.05):
    """
    ,,,This function is derived from code originally posted by Sat Kumar Tomer,,,(satkumartomer@gmail.com),,,See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm,,,The purpose of the Mann-Kendall (MK) test (Mann 1945, Kendall 1975, Gilbert,,,1987) is to statistically assess if there is a monotonic upward or downward,,,trend of the variable of interest over time. A monotonic upward (downward),,,trend means that the variable consistently increases (decreases) through,,,time, but the trend may or may not be linear. The MK test can be used in,,,place of a parametric linear regression analysis, which can be used to test,,,if the slope of the estimated linear regression line is different from,,,zero. The regression analysis requires that the residuals from the fitted,,,regression line be normally distributed; an assumption not required by the,,,MK test, that is, the MK test is a non-parametric (distribution-free) test.,,,Hirsch, Slack and Smith (1982, page 107) indicate that the MK test is best,,,viewed as an exploratory analysis and is most appropriately used to,,,identify stations where changes are significant or of large magnitude and,,,to quantify these findings.,,,Input:,,,x:   a vector of data,,,alpha: significance level (0.05 default),,,Output:,,,trend: tells the trend (increasing, decreasing or no trend),,,h: True (if trend is present) or False (if trend is absence),,,p: p value of the significance test,,,z: normalized test statistics,,,Examples,,,--------,,,>>> x = np.random.rand(100),,,>>> trend,h,p,z = mk_test(x,0.05)
    """
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z


def check_num_samples(beta, delta, std_dev, alpha=0.05, n=4, num_iter=1000,
                      tol=1e-6, num_cycles=10000, m=5):
    """
    ,,,This function is an implementation of the "Calculation of Number of Samples,,,Required to Detect a Trend" section written by Sat Kumar Tomer,,,(satkumartomer@gmail.com) which can be found at:,,,http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm,,,As stated on the webpage in the URL above the method uses a Monte-Carlo,,,simulation to determine the required number of points in time, n, to take a,,,measurement in order to detect a linear trend for specified small,,,probabilities that the MK test will make decision errors. If a non-linear,,,trend is actually present, then the value of n computed by VSP is only an,,,approximation to the correct n. If non-detects are expected in the,,,resulting data, then the value of n computed by VSP is only an,,,approximation to the correct n, and this approximation will tend to be less,,,accurate as the number of non-detects increases.,,,Input:,,,beta: probability of falsely accepting the null hypothesis,,,delta: change per sample period, i.e., the change that occurs between,,,two adjacent sampling times,,,std_dev: standard deviation of the sample points.,,,alpha: significance level (0.05 default),,,n: initial number of sample points (4 default).,,,num_iter: number of iterations of the Monte-Carlo simulation (1000,,,default).,,,tol: tolerance level to decide if the predicted probability is close,,,enough to the required statistical power value (1e-6 default).,,,num_cycles: Total number of cycles of the simulation. This is to ensure,,,that the simulation does finish regardless of convergence,,,or not (10000 default).,,,m: if the tolerance is too small then the simulation could continue to,,,cycle through the same sample numbers over and over. This parameter,,,determines how many cycles to look back. If the same number of,,,samples was been determined m cycles ago then the simulation will,,,stop.,,,Examples,,,--------,,,>>> num_samples = check_num_samples(0.2, 1, 0.1)
    """
    # Initialize the parameters
    power = 1.0 - beta
    P_d = 0.0
    cycle_num = 0
    min_diff_P_d_and_power = abs(P_d - power)
    best_P_d = P_d
    max_n = n
    min_n = n
    max_n_cycle = 1
    min_n_cycle = 1
    # Print information for user
    print("Delta (gradient): {}".format(delta))
    print("Standard deviation: {}".format(std_dev))
    print("Statistical power: {}".format(power))

    # Compute an estimate of probability of detecting a trend if the estimate
    # Is not close enough to the specified statistical power value or if the
    # number of iterations exceeds the number of defined cycles.
    while abs(P_d - power) > tol and cycle_num < num_cycles:
        cycle_num += 1
        print("Cycle Number: {}".format(cycle_num))
        count_of_trend_detections = 0

        # Perform MK test for random sample.
        for i in xrange(num_iter):
            r = np.random.normal(loc=0.0, scale=std_dev, size=n)
            x = r + delta * np.arange(n)
            trend, h, p, z = mk_test(x, alpha)
            if h:
                count_of_trend_detections += 1
        P_d = float(count_of_trend_detections) / num_iter

        # Determine if P_d is close to the power value.
        if abs(P_d - power) < tol:
            print("P_d: {}".format(P_d))
            print("{} samples are required".format(n))
            return n

        # Determine if the calculated probability is closest to the statistical
        # power.
        if min_diff_P_d_and_power > abs(P_d - power):
            min_diff_P_d_and_power = abs(P_d - power)
            best_P_d = P_d

        # Update max or min n.
        if n > max_n and abs(best_P_d - P_d) < tol:
            max_n = n
            max_n_cycle = cycle_num
        elif n < min_n and abs(best_P_d - P_d) < tol:
            min_n = n
            min_n_cycle = cycle_num

        # In case the tolerance is too small we'll stop the cycling when the
        # number of cycles, n, is cycling between the same values.
        elif (abs(max_n - n) == 0 and
              cycle_num - max_n_cycle >= m or
              abs(min_n - n) == 0 and
              cycle_num - min_n_cycle >= m):
            print("Number of samples required has converged.")
            print("P_d: {}".format(P_d))
            print("Approximately {} samples are required".format(n))
            return n

        # Determine whether to increase or decrease the number of samples.
        if P_d < power:
            n += 1
            print("P_d: {}".format(P_d))
            print("Increasing n to {}".format(n))
            print("")
        else:
            n -= 1
            print("P_d: {}".format(P_d))
            print("Decreasing n to {}".format(n))
            print("")
            if n == 0:
                raise ValueError("Number of samples = 0. This should not happen.")

In [5]:
btc_1h = get_all_bitmex('XBTUSD', '1h', save = False, start_time = datetime.strptime('01-01-2019 +0000', '%m-%d-%Y %z'))
btc_1h

1h
2020-04-05 04:00:00+00:00 2019-01-01 00:00:00+00:00



All caught up..!


,timestamp,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
0,2019-01-01 00:00:00+00:00,XBTUSD,3686.5,3695.5,3682.5,3693.0,10718,24965778,3689.7646,20,676636017934,6766.360179,24965778.0
1,2019-01-01 01:00:00+00:00,XBTUSD,3693.0,3705.0,3684.5,3694.0,9558,29678917,3695.0818,1102,803227026597,8032.270266,29678917.0
2,2019-01-01 02:00:00+00:00,XBTUSD,3694.0,3695.0,3675.5,3681.5,8563,18156574,3686.5000,2050,492520151894,4925.201519,18156574.0
3,2019-01-01 03:00:00+00:00,XBTUSD,3681.5,3683.5,3665.0,3678.5,9779,17395139,3676.6058,150,473140654578,4731.406546,17395139.0
4,2019-01-01 04:00:00+00:00,XBTUSD,3678.5,3687.0,3678.5,3685.0,6183,12509319,3683.9197,30000,339567556303,3395.675563,12509319.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,2020-04-05 00:00:00+00:00,XBTUSD,6857.0,6888.0,6835.0,6875.0,18529,33796685,6856.8294,6,492907121089,4929.071211,33796685.0
541,2020-04-05 01:00:00+00:00,XBTUSD,6875.0,6919.5,6850.0,6872.0,29684,57141858,6884.6816,100,830005322322,8300.053223,57141858.0
542,2020-04-05 02:00:00+00:00,XBTUSD,6872.0,6888.0,6840.0,6862.0,17553,33965425,6869.5473,100,494450778454,4944.507785,33965425.0
543,2020-04-05 03:00:00+00:00,XBTUSD,6862.0,6862.5,6806.0,6850.0,16097,32476562,6836.2045,5000,475068380365,4750.683804,32476562.0


In [7]:
trend,h,p,z = mk_test(btc_1h['close'].tail(1000).values,0.05)
trend

'decreasing'